In [15]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_core.messages import HumanMessage
from rag import make_react_agent_graph
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

In [16]:
settings = {
        "model": "gpt-4o-mini",
        "temperature": 0.5,
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "presence_penalty": 0,
    }

model = ChatOpenAI(
        model=settings["model"],
        temperature=settings["temperature"],
        # max_tokens=settings["max_tokens"] # ChatOpenAI might not take max_tokens directly here
    )
    
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

loader = PyMuPDFLoader("data/PracticalAdviceOnMatrixGames.pdf")
docs = loader.load()

split_chunks = text_splitter.split_documents(docs)
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
# Create a dummy collection. You'll need to populate this with actual documents for RAG to work.
vector_store = QdrantVectorStore.from_documents(
    split_chunks,
    embedding_function,
    location=":memory:",
    collection_name="matrix_game_docs",
)
# You might want to add some documents here if you have any, e.g.:
# vector_store.add_texts(["Some initial context for the agent"])

# Create the ReAct agent graph
# The search_kwargs for the vector store can be customized if needed
agent_graph = make_react_agent_graph(model=model, vector_store=vector_store, search_kwargs={"k": 3})

In [19]:
response = agent_graph.invoke({"messages": [HumanMessage(content="How are random events handled in matrix games?")]})
print(response)

{'messages': [HumanMessage(content='How are random events handled in matrix games?', additional_kwargs={}, response_metadata={}, id='ca91086f-70f4-4bbd-9a54-1f0a8b042d30'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mtPsEviVaOANhdWLf5VlaZXK', 'function': {'arguments': '{"query":"random events in matrix games"}', 'name': 'vector-search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BV6BdxWbCuApewT18XXgiWQ22tpFr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cfce4f7c-ac66-4a37-b703-bb9b335ab76c-0', tool_calls=[{'name': 'vector-sear

In [18]:
async for token, metadata in agent_graph.astream(
    {"messages": [{"role": "user", "content": "How are random events handled in matrix games?"}]},
    stream_mode="messages"
):
    print(token.content)
    print("metadata: ", metadata)


metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5}

metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f8099004c46d', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.5}

metadata:  {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ('branch:to:agent',), 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:da19a196-28cf-5dc3-ce6e-f809900

In [12]:
async for token, metadata in agent_graph.astream(
            {"messages": [HumanMessage(content="How are random events handled in matrix games?")]}, stream_mode = "messages"
        ):
            await print(token)
            await print(metadata)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'function': {'arguments': '', 'name': 'vector-search'}, 'type': 'function'}]} response_metadata={} id='run--d6a9582a-569c-415a-83c4-6c2bb868f99f' tool_calls=[{'name': 'vector-search', 'args': {}, 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'type': 'tool_call'}] tool_call_chunks=[{'name': 'vector-search', 'args': '', 'id': 'call_qah9XeCDgY6byTkPLwfBlanw', 'index': 0, 'type': 'tool_call_chunk'}]


TypeError: object NoneType can't be used in 'await' expression